In [1]:
import cogent
from numpy import allclose

import gapped

In [2]:
aln = cogent.LoadSeqs('../results/aligned_dna.fasta', moltype=cogent.DNA)
tree = cogent.LoadTree('../data/tree.nwk')

In [3]:
print tree.asciiArt()

                                        /-AmelF2
                              /twoBeesF2
                    /threeBeesF2        \-AdorF2
                   |         |
          /rootF2--|          \-BterF2
         |         |
         |         |          /-OsmaF2
         |          \twoAntsF2
         |                    \-MforF2
         |
         |                              /-AmelF3
         |                    /twoBeesF3
         |          /threeBeesF3        \-AdorF3
         |         |         |
         |-rootF3--|          \-BterF3
         |         |
-root----|         |          /-OsmaF3
         |          \twoAntsF3
         |                    \-MforF3
         |
         |                                        /-AmelF1
         |                              /twoBeesF1
         |                    /threeBeesF1        \-AdorF1
         |                   |         |
         |          /rootF1--|          \-BterF1
         |         |         |
         |

In [4]:
doc = {'aln' : str(aln), 'tree' : str(tree)}

In [5]:
cnfgtr_result = gapped.ml(doc, model='CNFGTR', model_gaps=True, omega_indep=False, indel_indep=False)

In [6]:
cnfgtr_result

{'gc': 'Standard Nuclear',
 'indel_indep': False,
 'lf': {'EN': {'AdorF1': 0.085159091671171847,
   'AdorF2': 0.10745171728078214,
   'AdorF3': 0.068736830232549007,
   'AdorF4': 0.033834933860977523,
   'AmelF1': 0.076812945996603904,
   'AmelF2': 0.063182466566373302,
   'AmelF3': 0.11347695829339546,
   'AmelF4': 0.1245657002543762,
   'BterF1': 0.29324806025815969,
   'BterF2': 0.57373414241832754,
   'BterF3': 0.48917244544265404,
   'BterF4': 1.0322387993917372,
   'MforF1': 0.89298237189780383,
   'MforF2': 0.76336337664023235,
   'MforF3': 0.60628984624119708,
   'MforF4': 0.77031283169263032,
   'OsmaF1': 0.43097894311847396,
   'OsmaF2': 0.67490401904204622,
   'OsmaF3': 0.5846349026281128,
   'OsmaF4': 0.87378160355798928,
   'rootF1': 0.60774757623232323,
   'rootF1F4': 0.40712289087758885,
   'rootF2': 0.69235688650179517,
   'rootF3': 1.1862730223403679,
   'rootF4': 0.12737966532679298,
   'threeBeesF1': 1.5517584288241184,
   'threeBeesF2': 1.369476316534064,
   'threeB

In [7]:
model = lambda: gapped.CNFGTR(optimise_motif_probs=True, model_gaps=True)
cnfgtr = gapped.inflate_likelihood_function(cnfgtr_result['lf'], model)

/Users/benkaehler/Projects/pycogent-bk/cogent/evolve/parameter_controller.py:241: UserWarning: Used mean of 38 mprobs values
  is_independent)
/Users/benkaehler/Projects/pycogent-bk/cogent/evolve/parameter_controller.py:241: UserWarning: Used mean of 37 C/T values
  is_independent)
/Users/benkaehler/Projects/pycogent-bk/cogent/evolve/parameter_controller.py:241: UserWarning: Used mean of 37 A/T values
  is_independent)
/Users/benkaehler/Projects/pycogent-bk/cogent/evolve/parameter_controller.py:241: UserWarning: Used mean of 37 indel values
  is_independent)
/Users/benkaehler/Projects/pycogent-bk/cogent/evolve/parameter_controller.py:241: UserWarning: Used mean of 37 C/G values
  is_independent)
/Users/benkaehler/Projects/pycogent-bk/cogent/evolve/parameter_controller.py:241: UserWarning: Used mean of 37 omega values
  is_independent)


In [8]:
for node in cnfgtr.tree.getEdgeVector(include_root=False):
    node.Length = cnfgtr.getParamValue('length', edge=node.Name)
cnfgtr.tree.writeToFile('../results/cnfgtr.nwk',with_distances=True)

In [9]:
print cnfgtr

Likelihood Function Table
   A/C       A/G       A/T       C/G       C/T     indel     omega
------------------------------------------------------------------
0.8812    2.3727    1.1745    0.8241    1.9841    0.1538    0.3385
------------------------------------------------------------------
       edge         parent    length
------------------------------------
     AmelF2      twoBeesF2    0.0632
     AdorF2      twoBeesF2    0.1075
  twoBeesF2    threeBeesF2    0.3895
     BterF2    threeBeesF2    0.5737
threeBeesF2         rootF2    1.3695
     OsmaF2      twoAntsF2    0.6749
     MforF2      twoAntsF2    0.7634
  twoAntsF2         rootF2    1.4453
     rootF2           root    0.6924
     AmelF3      twoBeesF3    0.1135
     AdorF3      twoBeesF3    0.0687
  twoBeesF3    threeBeesF3    0.5531
     BterF3    threeBeesF3    0.4892
threeBeesF3         rootF3    1.0477
     OsmaF3      twoAntsF3    0.5846
     MforF3      twoAntsF3    0.6063
  twoAntsF3         rootF3    1.2479
   

In [10]:
raw_length = len(aln)
aln = aln.withoutTerminalStopCodons()
filt = lambda x: set(''.join(x)) <= set(cogent.DNA).union({'-'})
aln = aln.filtered(filt, motif_length=3)
assert len(aln) == raw_length
cnfgtr.setAlignment(aln)

In [11]:
anc_aln = gapped.joint(cnfgtr)

In [12]:
anc_aln.writeToFile('../results/joint.fasta')